In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product

## Questão 1: Arranha-céus!

### a)

### b)

### c)



## Questão 2: Plantação de tomates!

### a) Estratégia gulosa
A estratégia gulosa funciona para o exemplo simples onde o jardim é dado por

| 5 | 3 | 5 |

Um contra exemplo à essa estratégia é o jardim 

| 3 | 5 | 3 |

Nesse último caso, a estratégia gulosa nos dá uma delicioncisade total de 5, sendo que o ótimo é na realidade 6.

### b)

### c) Programa linear

\begin{align*}
    \text{Maximizar }   & \sum_{i=1}^{n} T_i x_i \hspace{20pt} \text{(Deliciosidade total)}\\
    \text{sujeito a }   & \; x_i + x_{i+1} = 1 \quad \forall i \in \{1,..., n-1\}  \hspace{20pt} \text{(Não plantar dois tomates em casas adjacentes)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - plantar ou não plantar tomate em cada casa)} \\
                        & T_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Deliciosidades inteiras e positivas)} \\
\end{align*}

### d)



## Questão 3: Dois problemas fundamentais em grafos bipartidos

### a)

### b)

### c)

### d)



## Questão 4: O Princípio da Casa dos Pombos

### a)

### b)

### c)



## Questão 5: Pudim, o pinguim comilão.

### a)
O problema só é factível se não houver algum segmento da viagem de comprimento maior de que $m$, a distância máxima que Pudim pode viajar sem comer. Mais formalmente, o problema é factível se

$F_i - F_{i-1} \leq m \quad \forall i \in \{ 2,...,n \}$

### b)

\begin{align*}
    \text{Minimizar }   & \sum_{i=1}^{n} x_i \hspace{20pt} \text{(Total de paradas realizadas)}\\
    \text{sujeito a }   & \; x_i + x_{i+1} = 1 \quad \forall i \in \{1,..., n-1\}  \hspace{20pt} \text{(Não plantar dois tomates em casas adjacentes)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - parar ou não no buraco de pesca)} \\
                        & F_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Distâncias inteiras e positivas)} \\
\end{align*}

### c)